In [ ]:
import requests

In [ ]:
import json
from importlib import reload
import fxcm_rest_api_token as fxcm_rest_api
result = reload(fxcm_rest_api)

In [ ]:
trader = fxcm_rest_api.Trader('YOURKEYHERE', 'prod')
trader.debug_level = "INFO" # verbose logging... don't set to receive errors only
trader.login()

In [ ]:
c =trader.candles("EUR/USD", "m15", 45, dt_fmt="%Y/%m/%d %H:%M:%S")['candles']
print(len(c))
for candle in c:
    print(candle)

In [ ]:
help(trader.open_trade)

In [ ]:
res = trader.open_trade(trader.account_id, "USD/JPY", True, 1)
orderId = res['data']['orderId']

In [ ]:
tradeId = trader.get_tradeId(orderId)

In [ ]:
print(tradeId)

In [ ]:
trader.close_all_for_symbol("USD/JPY")

In [ ]:
def print_candles(candle_data):
    candles = candle_data['candles']
    headers = candle_data['headers']
    print("{0[10]}, {0[1]}, {0[3]}, {0[4]}, {0[2]}".format(headers))
    candles.reverse()
    for candle in candles:
        print("{0[10]}, {0[1]:0.6}, {0[3]:0.6}, {0[4]:0.6}, {0[2]:0.6}".format(candle))

In [ ]:
print_candles(cd)

In [ ]:
print_candles(trader.get_candles("EUR/USD", "m1", 5, dt_fmt="%Y/%m/%d %H:%M:%S"))
# time.sleep(60)
# print("*"*30)
# print_candles(trader.get_candles("EUR/USD", "m1", 5, dt_fmt="%Y/%m/%d %H:%M:%aS"))

In [ ]:
trader.candles_as_dict("EUR/USD", "m1", 1, dt_fmt="%Y/%m/%d %H:%M:%S")



In [ ]:
trader.close_all_for_symbol(trader.account_id, True, "USD/JPY", "AtMarket", "GTC")

In [ ]:
trader.accounts

In [ ]:
trader.properties

In [ ]:
print(trader.account_id)

#### Show how to supply a different handler for subscription items

In [ ]:

def show(msg):
    '''
    Sample price handler. If on_price_update is registered for a symbol,
    it will update the symbol's values (stored in a symbol hash) with
    that price update.symbol hash.

    :return: none
    '''
    try:
        md = json.loads(msg)
        symbol = md["Symbol"]
        t = trader
        si = trader.symbol_info.get(symbol, {})
        p_up = dict(symbol_info=t.symbol_info[symbol], parent=t)
        t.symbols[symbol] = t.symbols.get(symbol, fxcm_rest_api
                                          .PriceUpdate(p_up,
                                                       symbol_info=si))
        trader.symbols[symbol].bid, trader.symbols[symbol].ask,\
            trader.symbols[symbol].high,\
            trader.symbols[symbol].low = md['Rates']
        trader.symbols[symbol].updated = md['Updated']
        print(t.symbols[symbol])
    except Exception as e:
        trader.logger.error("Can't handle price update: " + str(e))

# trader = fx

In [ ]:
#trader.unsubscribe_symbol("EUR/USD")
trader.subscribe_symbol("EUR/USD", handler=show)

In [ ]:
trader.unsubscribe_symbol("EUR/USD")

In [ ]:
result = trader.subscribe_symbol("GBP/JPY")
print(result)
print(trader.subscriptions)

In [ ]:
print(trader.symbols["GBP/JPY"])
print(trader.symbols["GBP/JPY"].bid)
print(trader.symbols["GBP/JPY"].ask)

In [ ]:
print(trader.symbols['GBP/JPY'].parent)

In [ ]:
trader.unsubscribe_symbol("GBP/JPY")

In [ ]:
trader.open_trade(trader.account_id, "USD/JPY", True, 1, at_market=1, time_in_force="FOK")